Create .env file in the same directory as the notebook and add the following lines:

```env
USERNAME=your_username
PASSWORD=your_password
ENDPOINT=your_endpoint
```

Install python dependencies.

In [ ]:
pip install -r requirements.txt

Connect to database.

In [ ]:
import os
from graphdatascience       import GraphDataScience
from dotenv                 import load_dotenv

load_dotenv()

gds = GraphDataScience(os.environ["ENDPOINT"],
                       auth=(os.environ["USERNAME"], os.environ["PASSWORD"]))

gds.set_database("neo4j")
print(gds.version())

Should show a version number if connection to database is successful.